# **AB Testi ile Bidding Yöntemlerinin Dönüşümünün Karşılaştırılması**


##İş Problemi:
Facebook kısa süre önce mevcut "maximum bidding" adı verilen
teklif verme türüne alternatif olarak yeni bir teklif türü olan
"average bidding"’i tanıttı.

Müşterilerimizden biri olan bombabomba.com, bu yeni özelliği test
etmeye karar verdi ve average bidding'in maximum bidding'den
daha fazla dönüşüm getirip getirmediğini anlamak için bir A/B
testi yapmak istiyor.

A/B testi 1 aydır devam ediyor ve bombabomba.com şimdi sizden
bu A/B testinin sonuçlarını analiz etmenizi bekliyor.
Bombabomba.com için nihai başarı ölçütü Purchase'dır. Bu
nedenle, istatistiksel testler için Purchase metriğine
odaklanılmalıdır.


## Veri Seti Hikayesi:
Bir firmanın web site bilgilerini içeren bu veri setinde kullanıcıların gördükleri ve tıkladıkları reklam sayıları gibi bilgilerin yanı sıra
buradan gelen kazanç bilgileri yer almaktadır. Kontrol ve Test grubu olmak üzere iki ayrı veri seti vardır. Bu veri setleri
ab_testing.xlsx excel’inin ayrı sayfalarında yer almaktadır. Kontrol grubuna Maximum Bidding, test grubuna Average
Bidding uygulanmıştır.


## Görev 1: Veriyi Hazırlama ve Analiz Etme


In [4]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
!pip install statsmodels
import statsmodels.stats.api as sms
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu, pearsonr, spearmanr, kendalltau, f_oneway, kruskal
from statsmodels.stats.proportion import proportions_ztest

In [5]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", False)
pd.set_option("display.float_format", lambda x:"%.5f" %x)

### Adım 1: ab_testing_data.xlsx adlı kontrol ve test grubu verilerinden oluşan veri setini okutunuz. Kontrol ve test grubu verilerini ayrı değişkenlere atayınız.

In [6]:
dataframe_control = pd.read_excel("/content/ab_testing.xlsx",0)
dataframe_control.head()

,Impression,Click,Purchase,Earning
0,82529.45927,6090.07732,665.21125,2311.27714
1,98050.45193,3382.86179,315.08489,1742.80686
2,82696.02355,4167.96575,458.08374,1797.82745
3,109914.40040,4910.88224,487.09077,1696.22918
4,108457.76263,5987.65581,441.03405,1543.72018


In [7]:
dataframe_test = pd.read_excel("/content/ab_testing.xlsx",1)
dataframe_test.head()

,Impression,Click,Purchase,Earning
0,120103.50380,3216.54796,702.16035,1939.61124
1,134775.94336,3635.08242,834.05429,2929.40582
2,107806.62079,3057.14356,422.93426,2526.24488
3,116445.27553,4650.47391,429.03353,2281.42857
4,145082.51684,5201.38772,749.86044,2781.69752


In [8]:
df_control = dataframe_control.copy()
df_test = dataframe_test.copy()

### Adım 2: Kontrol ve test grubu verilerini analiz ediniz.

In [9]:
def check_df(dataframe, head=5):
  print("########## SHAPE ##########")
  print(dataframe.shape)
  print("########## TYPES ##########")
  print(dataframe.dtypes)
  print("########## HEAD ##########")
  print(dataframe.head(head))
  print("########## TAIL ##########")
  print(dataframe.tail(head))
  print("########## NA ##########")
  print(dataframe.isnull().sum())
  print("########## QUANTILES ##########")
  print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

In [10]:
check_df(df_control)

########## SHAPE ##########
(40, 4)
########## TYPES ##########
Impression    float64
Click         float64
Purchase      float64
Earning       float64
dtype: object
########## HEAD ##########
    Impression      Click  Purchase    Earning
0  82529.45927 6090.07732 665.21125 2311.27714
1  98050.45193 3382.86179 315.08489 1742.80686
2  82696.02355 4167.96575 458.08374 1797.82745
3 109914.40040 4910.88224 487.09077 1696.22918
4 108457.76263 5987.65581 441.03405 1543.72018
########## TAIL ##########
     Impression      Click  Purchase    Earning
35 132064.21900 3747.15754 551.07241 2256.97559
36  86409.94180 4608.25621 345.04603 1781.35769
37 123678.93423 3649.07379 476.16813 2187.72122
38 101997.49410 4736.35337 474.61354 2254.56383
39 121085.88122 4285.17861 590.40602 1289.30895
########## NA ##########
Impression    0
Click         0
Purchase      0
Earning       0
dtype: int64
########## QUANTILES ##########
               0.00000     0.05000     0.50000      0.95000      0.99000  \


In [11]:
# 0.99-1.00 arasında büyük bir sıçrama olmadığı için aykırı değer yoktur yorumunu yapabiliriz.

In [12]:
check_df(df_test)

########## SHAPE ##########
(40, 4)
########## TYPES ##########
Impression    float64
Click         float64
Purchase      float64
Earning       float64
dtype: object
########## HEAD ##########
    Impression      Click  Purchase    Earning
0 120103.50380 3216.54796 702.16035 1939.61124
1 134775.94336 3635.08242 834.05429 2929.40582
2 107806.62079 3057.14356 422.93426 2526.24488
3 116445.27553 4650.47391 429.03353 2281.42857
4 145082.51684 5201.38772 749.86044 2781.69752
########## TAIL ##########
     Impression      Click  Purchase    Earning
35  79234.91193 6002.21358 382.04712 2277.86398
36 130702.23941 3626.32007 449.82459 2530.84133
37 116481.87337 4702.78247 472.45373 2597.91763
38  79033.83492 4495.42818 425.35910 2595.85788
39 102257.45409 4800.06832 521.31073 2967.51839
########## NA ##########
Impression    0
Click         0
Purchase      0
Earning       0
dtype: int64
########## QUANTILES ##########
               0.00000     0.05000      0.50000      0.95000      0.99000  \

In [13]:
# 0.99-1.00 arasında büyük bir sıçrama olmadığı için aykırı değer yoktur yorumunu yapabiliriz.

### Adım 3: Analiz işleminden sonra concat metodunu kullanarak kontrol ve test grubu verilerini birleştiriniz.

In [14]:
df_control["group"] = "control"
df_test["group"] = "test"

In [15]:
df = pd.concat([df_control, df_test], axis=0, ignore_index=True)

In [16]:
df.head()

,Impression,Click,Purchase,Earning,group
0,82529.45927,6090.07732,665.21125,2311.27714,control
1,98050.45193,3382.86179,315.08489,1742.80686,control
2,82696.02355,4167.96575,458.08374,1797.82745,control
3,109914.40040,4910.88224,487.09077,1696.22918,control
4,108457.76263,5987.65581,441.03405,1543.72018,control


In [17]:
df.tail()

,Impression,Click,Purchase,Earning,group
75,79234.91193,6002.21358,382.04712,2277.86398,test
76,130702.23941,3626.32007,449.82459,2530.84133,test
77,116481.87337,4702.78247,472.45373,2597.91763,test
78,79033.83492,4495.42818,425.35910,2595.85788,test
79,102257.45409,4800.06832,521.31073,2967.51839,test


## Görev 2: A/B Testinin Hipotezinin Tanımlanması


### Adım 1: Hipotezi tanımlayınız.

In [18]:
# H0 : M1 = M2 (Kontrol grubu ve test grubu satın alma ortalamaları arasında fark yoktur.)
# H1 : M1!= M2 (Kontrol grubu ve test grubu satın alma ortalamarı arasında fark vardır.)

### Adım 2: Kontrol ve test grubu için purchase (kazanç) ortalamalarını analiz ediniz.

In [19]:
df.groupby("group").agg({"Purchase":"mean"})

,Purchase
group,
control,550.89406
test,582.10610


In [20]:
# Herhangi bir istatistiki metoda başvurmadan incelendiğinde iki grup arasında fark var yorumunu yapabiliriz.

## Görev 3: Hipotez Testinin Gerçekleştirilmesi


### Adım 1: Hipotez testi yapılmadan önce varsayım kontrollerini yapınız. Bunlar Normallik Varsayımı ve Varyans Homojenliğidir. Kontrol ve test grubunun normallik varsayımına uyup uymadığını Purchase değişkeni üzerinden ayrı ayrı test ediniz.

Normallik Varsayımı :

H0: Normal dağılım varsayımı sağlanmaktadır.

H1: Normal dağılım varsayımı sağlanmamaktadır.

p < 0.05 H0 RED , p > 0.05 H0 REDDEDİLEMEZ

Test sonucuna göre normallik varsayımı kontrol ve test grupları için sağlanıyor mu ? Elde edilen p-value değerlerini yorumlayınız.

In [21]:
test_stat, pvalue = shapiro(df.loc[df["group"]=="control","Purchase"])
print("Test Stat = %.4f, p-value = %.4f" % (test_stat, pvalue))

Test Stat = 0.9773, p-value = 0.5891


In [ ]:
# HO Reddedilemez. Control grubunun değerleri normal dağılım varsayımını sağlamaktadır.

In [22]:
test_stat, pvalue = shapiro(df.loc[df["group"]=="test","Purchase"])
print("Test Stat = %.4f, p-value = %.4f" % (test_stat, pvalue))

Test Stat = 0.9589, p-value = 0.1541


In [ ]:
# HO Reddedilemez. Test grubunun değerleri normal dağılım varsayımını sağlamaktadır.

Varyans Homojenliği :

H0: Varyanslar homojendir.

H1: Varyanslar homojen Değildir.

p < 0.05 H0 RED , p > 0.05 H0 REDDEDİLEMEZ

Kontrol ve test grubu için varyans homojenliğinin sağlanıp sağlanmadığını Purchase değişkeni üzerinden test ediniz.
Test sonucuna göre normallik varsayımı sağlanıyor mu? Elde edilen p-value değerlerini yorumlayınız.

In [23]:
test_stat, pvalue = levene(df.loc[df["group"]=="control","Purchase"],
                           df.loc[df["group"]=="test","Purchase"])
print("Test Stat = %.4f, p-value = %.4f" % (test_stat, pvalue))

Test Stat = 2.6393, p-value = 0.1083


In [ ]:
# H0 Reddedilemez. Control ve Test grubunun değerleri varyans homojenliği varsayımını sağlamaktadır.

### Adım 2: Normallik Varsayımı ve Varyans Homojenliği sonuçlarına göre uygun testi seçiniz.

Varsayımlar sağlandığı için Bağımsız İki Örneklem T Testi (parametrik test) yapılmaktadır.

Parametrik Test:

H0 : M1 = M2 (Kontrol grubu ve test grubu satın alma ortalamaları arasında istatistiksel olarak anlamlı bir fark yoktur.)

H1 : M1!= M2 (Kontrol grubu ve test grubu satın alma ortalamarı arasında istatistiksel olarak anlamlı bir fark vardır.)

p < 0.05 H0 RED , p > 0.05 H0 REDDEDİLEMEZ

In [25]:
test_stat, pvalue = ttest_ind(df.loc[df["group"]=="control","Purchase"],
                              df.loc[df["group"]=="test","Purchase"],
                              equal_var=True)
print("Test Stat = %.4f, p-value = %.4f" % (test_stat, pvalue))

Test Stat = -0.9416, p-value = 0.3493


### Adım 3: Test sonucunda elde edilen p_value değerini göz önünde bulundurarak kontrol ve test grubu satın alma ortalamaları arasında istatistiki olarak anlamlı bir fark olup olmadığını yorumlayınız.

In [ ]:
# HO Reddedilemez. Control ve Test grubu satın alma ortalamaları arasında istatistiksel olarak anlamlı bir farklılık yoktur.

## Görev 4: Sonuçların Analizi


### Adım 1: Hangi testi kullandınız, sebeplerini belirtiniz.

İlk önce iki gruba da normallik testi uygulanmıştır. İki grubunda normal dağılıma uyduğu gözlemlendiğinden ikinci varsayıma geçilerek varyansın homojenliği incelenmiştir. Varyanslar homojen çıktığından Bağımsız İki Örneklem T Testi uygulanmıştır. Uygulama sonucunda p-değerinin 0.05'ten büyük olduğu görülmüş ve H0 hipotezi reddedilememiştir.

### Adım 2: Elde ettiğiniz test sonuçlarına göre müşteriye tavsiyede bulununuz

Ortalamadaki farklılıklar şans eseri oluşmuştur. Satın alma anlamında anlamlı bir fark olmadığından müşteri iki yöntemden birini seçebilir fakat burada diğer istatistiklerdeki farklılıklar da önem arz edecektir. Tıklanma, etkileşim, kazanç ve dönüşüm oranlarındaki farklılıklar değerlendirilip hangi yöntemin daha kazançlı olduğu tespit edilebilir. Özellikle Facebook'a tıklama başına para ödendiği için hangi yöntemde tıklanma oranının daha düşük olduğu tespit edilip ve CTR(Click Through Rate - Tıklama Oranı) oranına bakılabilir.